## 任务 2：训练模型

在之前的任务中，我们使用了内置的 SageMaker XGBoost 算法。这种 XGBoost 内置算法模型不包含您自己的 XGBoost 训练脚本，而是直接在输入数据集上运行。

在本任务中，我们将使用 XGBoost 作为框架。通过在 SageMaker 中将 XGBoost 作为框架运行，您可以更灵活地访问更高级的场景，例如 K 折交叉验证，因为您可以自定义自己的训练脚本。

除了使用训练脚本运行训练作业外，我们还将通过运行超参数优化作业来优化训练。超参数是影响模型训练期间学习过程的高级参数。要获得最佳的模型预测，您可以优化超参数配置或设置超参数范围。寻找最佳配置的过程称为超参数调整。


### 任务 2.1：设置环境

在开始训练模型之前，请安装所有必要依赖项。


In [ ]:
import pandas as pd
import boto3
import sagemaker
import json
import joblib
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.tuner import (
    IntegerParameter,
    ContinuousParameter,
    HyperparameterTuner
)
from sagemaker.inputs import TrainingInput
from sagemaker.image_uris import retrieve
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

# Setting SageMaker variables
sess = sagemaker.Session()
region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)
sagemaker_role = sagemaker.get_execution_role()
sagemaker_client = boto3.client("sagemaker")

 接下来，和之前一样，我们要配置训练作业用作输入的训练路径和验证路径。

In [ ]:
# define the bucket and prefix
#read_bucket = "<LAB_BUCKET>"
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        read_bucket = buckets.name
#read_bucket = "labdatabucket-us-west-2-110972467"
read_prefix = "scripts/data" 

# Setting S3 location for read and write operations
train_data_key = f"{read_prefix}/train/adult_data_processed_train_wheaders.csv"
test_data_key = f"{read_prefix}/test/adult_data_processed_test_wheaders.csv"
validation_data_key = f"{read_prefix}/validation/adult_data_processed_validation_wheaders.csv"

write_bucket = read_bucket
write_prefix = "script-mode/data"

model_key = f"{write_prefix}/model"
output_key = f"{write_prefix}/output"

train_data_uri = f"s3://{read_bucket}/{train_data_key}"
validation_data_uri = f"s3://{read_bucket}/{validation_data_key}"
test_data_uri = f"s3://{read_bucket}/{test_data_key}"
model_uri = f"s3://{write_bucket}/{model_key}"
output_uri = f"s3://{write_bucket}/{output_key}"
estimator_output_uri = f"s3://{write_bucket}/{write_prefix}/training_jobs"
bias_report_output_uri = f"s3://{write_bucket}/{write_prefix}/clarify-output/bias"
explainability_report_output_uri = f"s3://{write_bucket}/{write_prefix}/clarify-output/explainability"

### 任务 2.2：配置估算器对象

调整作业配置 

In [ ]:
training_job_name_prefix = "xgbtrain"
tuning_job_name_prefix = "xgbtune" 
xgb_model_name = "xgb-script-mode-model"
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

配置静态超参数、超参数范围和估算器对象。

请注意，与任务 1 不同，在此脚本模式任务中，我们为训练作业定义自定义脚本。请注意，在下面定义 SageMaker 估算器时，我们使用该自定义的 Python 脚本作为入口点。花点时间打开自定义 xgboost_train.py 文件。

在自定义 xgboost_train.py 文件中，您将看到我们执行交叉验证技术的部分。使用即装即用的 SageMaker XGBoost 训练算法无法做到这一点。

另请注意，我们在下一个单元格中定义的超参数范围与我们在第一个任务的 XGBoost 内置训练中使用的静态超参数相同，但是这次我们定义的是范围，让优化器尝试这些超参数的不同值，以找到具有最佳最终目标指标的组合。


In [ ]:
# Set static hyperparameters that will not be tuned
static_hyperparams = {  
                        "eval_metric" : "auc",
                        "objective": "binary:logistic",
                        "num_round": "5"
                      }

# hyperparameter ranges that will be tuned 
hyperparameter_ranges = {
    "max_depth": IntegerParameter(6, 9),
    "eta": ContinuousParameter(0.01, 0.03),
    "gamma": ContinuousParameter(0.5, 0.9),
    "min_child_weight": ContinuousParameter(0.5, 0.9),
    "subsample": ContinuousParameter(0.2, 0.5)
}

# XGBoost Estimator 
xgb_estimator = XGBoost(
                        entry_point="xgboost_train.py",
                        output_path=estimator_output_uri,
                        code_location=estimator_output_uri,
                        hyperparameters=static_hyperparams,
                        role=sagemaker_role,
                        instance_count=train_instance_count,
                        instance_type=train_instance_type,
                        framework_version="1.7-1",
                        base_job_name=training_job_name_prefix
                    )

现在，创建一个优化器对象，该对象将使用 XGBoost 估算器以及我们定义的超参数范围。

In [ ]:
objective_metric_name = "validation:auc"

# Setting up tuner object
tuner_config_dict = {
                     "estimator" : xgb_estimator,
                     "max_jobs" : 6,
                     "max_parallel_jobs" : 3,    
                     "objective_metric_name" : objective_metric_name,
                     "hyperparameter_ranges" : hyperparameter_ranges,
                     "base_tuning_job_name" : tuning_job_name_prefix,
                     "strategy" : "Random"
                    }
tuner = HyperparameterTuner(**tuner_config_dict)

为优化作业设置输入通道并运行优化器作业 

In [ ]:
s3_input_train = TrainingInput(s3_data="s3://{}/{}".format(read_bucket, train_data_key), content_type="csv", s3_data_type="S3Prefix")
s3_input_validation = (TrainingInput(s3_data="s3://{}/{}".format(read_bucket, validation_data_key), content_type="csv", s3_data_type="S3Prefix"))

tuner.fit(inputs={"train": s3_input_train, "validation": s3_input_validation}, include_cls_metadata=False)
tuner.wait()

<i aria-hidden="true" class="fas fa-clipboard-check" style="color:#18ab4b"></i>**预期输出**：如果估算器和超参数配置正确，且超参数调整作业正确启动，您应该能看到以下输出：

```plain
************************
**** EXAMPLE OUTPUT ****
************************

No finished training job found associated with this estimator.Please make sure this estimator is only used for building workflow config
...................................!
!
```

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#ff6633"></i>**注意**：运行训练大约需要 3–4 分钟。


运行按性能降序排列的调整结果摘要，其中目标值最高的排在最前面。 

In [ ]:
df_tuner = sagemaker.HyperparameterTuningJobAnalytics(tuner.latest_tuning_job.job_name).dataframe()
df_tuner = df_tuner[df_tuner["FinalObjectiveValue"]>-float('inf')].sort_values("FinalObjectiveValue", ascending=False)
df_tuner

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件。
- 返回至实验会话并结束实验。